# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [32]:
# import libraries
import nltk
from nltk.stem.porter import PorterStemmer
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import re
from sklearn.pipeline import Pipeline, FeatureUnion
from nltk.tokenize import word_tokenize
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from nltk.corpus import stopwords
from sklearn.model_selection import GridSearchCV
import pickle

In [33]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table("Messages", engine)
X = df['message']
Y = df.drop(columns =['id', 'message','original','genre'])

In [ ]:
y

In [35]:
X

0        Weather update - a cold front from Cuba that c...
1                  Is the Hurricane over or is it not over
2                          Looking for someone but no name
3        UN reports Leogane 80-90 destroyed. Only Hospi...
4        says: west side of Haiti, rest of the country ...
                               ...                        
26211    The training demonstrated how to enhance micro...
26212    A suitable candidate has been selected and OCH...
26213    Proshika, operating in Cox's Bazar municipalit...
26214    Some 2,000 women protesting against the conduc...
26215    A radical shift in thinking came about as a re...
Name: message, Length: 26216, dtype: object

### 2. Write a tokenization function to process your text data

In [36]:
def tokenize(text):
    """
    The functions takes text as input and tokenizes it by removing punctuations, converting all letters to lower case,
    removing the stop words and reducing it to its stems.

    :param text: the text to be tokenized.
    :return: tokenized text .
    """

    # Remove punctuation characters
    text = re.sub(r"[^a-zA-Z0-9]", " ", str(text))
    
    # Convert to lowercase
    try:
        text = text.str.lower()
    except: 
        text=text.lower()

    #tokenize
    words = word_tokenize(text)
    
    # #remove stopwords
    words = [w for w in words if w not in stopwords.words("english")]

    #reduce words to their stems
    stemmed = [PorterStemmer().stem(w) for w in words]
    
    return stemmed

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [37]:
#Building a ML pipeline
pipeline = Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [38]:
#Spliting the data into the train and test for the machine learning model
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

In [39]:
#fitting the data into the pipeline
pipeline.fit(X_train, Y_train)

C:\Users\mkaud\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x0000021A3CEE8220>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

In [40]:
Y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [63]:
def model_report():
    """
    The function iterates through all the columns and prints out the precision, recall and f1 score for each using classification_report.
    
    """
    cols = df[df.columns[4:]].columns

    i=0
    for col in cols:
        report = classification_report(Y_test[col], Y_pred[:, i])
        print(f"Classification report for column '{col}':\n{report}\n")
        i +=1
        
    accuracy = (Y_pred == Y_test).mean()
    print("Accuracy:\n", accuracy)

In [ ]:
model_report()

### 6. Improve your model
Use grid search to find better parameters. 

In [44]:
#list the available parameters for Pipeline
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x0000021A3CEE8220>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x0000021A3CEE8220>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,


In [49]:
#Use the parameter to tune/improve the model
parameters = {
        'clf__estimator__n_estimators': [50, 100],
}

#Initialize the GridSearchCV and tune the pipeline with the provided parameters
cv = GridSearchCV(pipeline, param_grid=parameters, cv=3)


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [50]:
#fit the data into the tuning model
cv.fit(X_train, Y_train)

C:\Users\mkaud\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\mkaud\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\mkaud\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\mkaud\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\mkaud\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_extraction\text.py:525: Use

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x0000021A3CEE8220>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__n_estimators': [50, 100]})

In [52]:
Y_pred = cv.predict(X_test)

In [53]:
#list the best parameters of the GridSearch
cv.best_params_

{'clf__estimator__n_estimators': 50}

In [62]:
model_report()

Classification report for column 'related':
              precision    recall  f1-score   support

           0       0.72      0.43      0.54      1537
           1       0.84      0.95      0.89      4969
           2       0.60      0.12      0.21        48

    accuracy                           0.82      6554
   macro avg       0.72      0.50      0.55      6554
weighted avg       0.81      0.82      0.80      6554


Classification report for column 'request':
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5439
           1       0.84      0.48      0.61      1115

    accuracy                           0.90      6554
   macro avg       0.87      0.73      0.77      6554
weighted avg       0.89      0.90      0.88      6554


Classification report for column 'offer':
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6521
           1       0.00      0.00      0.00        

C:\Users\mkaud\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\mkaud\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\mkaud\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

Classification report for column 'electricity':
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6416
           1       0.86      0.04      0.08       138

    accuracy                           0.98      6554
   macro avg       0.92      0.52      0.54      6554
weighted avg       0.98      0.98      0.97      6554


Classification report for column 'tools':
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6510
           1       0.00      0.00      0.00        44

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.99      0.99      6554


Classification report for column 'hospitals':
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6490
           1       0.00      0.00      0.00        64

    accuracy                           0.99 

C:\Users\mkaud\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\mkaud\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\mkaud\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [58]:
#Export the model to a pickle file.
pickle_file_path = "model.pkl"

with open(pickle_file_path, "wb") as pickle_file:
    pickle.dump(pipeline, pickle_file)

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.